# Fine-tune model with DPO

## Goal

Let's see if using DPO can create a better model.

## Imports

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import pandas as pd

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
)

from trl import DPOConfig, DPOTrainer
from datasets import Dataset

from peft import LoraConfig

## Load model

In [3]:
model_path = '/home/gbarbadillo/data/deepseekmath'
config = AutoConfig.from_pretrained(model_path)
config.gradient_checkpointing = True

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    torch_dtype="auto", #torch.bfloat16 does not show speed differences
    trust_remote_code=True,
    quantization_config=None,
    config=config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# TODO: check pad token on prompt recovery notebook
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Load data

In [10]:
df = pd.read_csv('/mnt/hdd0/Kaggle/aimo/external_data/dpo/v1.csv')
df.head()

,prompt,chosen,rejected,problem_idx,max_prompt_length,chosen_length,rejected_length
0,User: John computes the sum of the elements of...,from sympy import *\n\ndef sum_of_subset_sums(...,from itertools import combinations\n\ndef sum_...,0,92,166,159
1,You are an expert mathematical programmer. Sol...,from itertools import combinations\n\ndef sum_...,from sympy import binomial\n\ndef sum_of_subse...,0,147,168,199
2,\nUser: John computes the sum of the elements ...,from itertools import combinations\n\ndef sum_...,"from sympy import binomial, Rational, simplify...",0,156,118,405
3,User: John computes the sum of the elements of...,from itertools import combinations\n\ndef sum_...,"from sympy import binomial, summation, symbols...",0,99,179,459
4,Below is a math problem you are to solve (non ...,from itertools import combinations\n\ndef sum_...,from itertools import combinations\n\ndef sum_...,0,100,163,189


In [11]:
unique_problem_ids = df['problem_idx'].unique()
len(unique_problem_ids)

509

In [14]:
np.random.seed(7)
train_problem_ids = np.random.choice(unique_problem_ids, int(0.95 * len(unique_problem_ids)), replace=False)
train_df = df[df['problem_idx'].isin(train_problem_ids)]
test_df = df[~df['problem_idx'].isin(train_problem_ids)]
assert len(train_df) + len(test_df) == len(df)
assert set(train_df['problem_idx'].unique()).intersection(set(test_df['problem_idx'].unique())) == set()
print(len(train_df), len(test_df))

10217 444


In [15]:
# shuffle the train dataset
train_dataset = Dataset.from_pandas(train_df[['prompt', 'chosen', 'rejected']].sample(frac=1))
eval_dataset = Dataset.from_pandas(test_df[['prompt', 'chosen', 'rejected']])

## Fine-tuning

In [9]:
raise

RuntimeError: No active exception to reraise

### First experiment

In [ ]:
from peft import LoraConfig

# LoRA config based on https://github.com/ironbar/prompt_recovery/blob/main/notebooks/014_fine-tune_mistral_v2.ipynb
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05, # 0.1, althought Vaca suggested to use 0.05 for big models
        # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters
        r=16,
        bias="none",
        # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [ ]:
# https://www.philschmid.de/dpo-align-llms-in-2024-with-trl
# https://huggingface.co/docs/trl/main/en/dpo_trainer
args = DPOConfig(
    output_dir="/mnt/hdd0/Kaggle/aimo/experiments/18_dpo/01_dpo-first-steps",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,         # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    per_device_eval_batch_size=4,           # batch size for evaluation
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=2e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=10,
    save_steps=100,                         # when to save checkpoint
    # save_total_limit=2,                     # limit the total amount of checkpoints
    eval_strategy="steps",                  # evaluate every n steps
    eval_steps=100,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    model_init_kwargs=None,
    max_length=1024,
    max_prompt_length=300,
    beta=0.1,                               # same as huggingface documentation, default value
    loss_type="sigmoid",                    # default value in huggingface documentation
)

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

### 2. Shuffle the data

In [ ]:
from peft import LoraConfig

# LoRA config based on https://github.com/ironbar/prompt_recovery/blob/main/notebooks/014_fine-tune_mistral_v2.ipynb
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05, # 0.1, althought Vaca suggested to use 0.05 for big models
        # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters
        r=16,
        bias="none",
        # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [ ]:
# https://www.philschmid.de/dpo-align-llms-in-2024-with-trl
# https://huggingface.co/docs/trl/main/en/dpo_trainer
args = DPOConfig(
    output_dir="/mnt/hdd0/Kaggle/aimo/experiments/18_dpo/02_shuffle_train_set",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,         # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    per_device_eval_batch_size=4,           # batch size for evaluation
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=2e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=10,
    save_steps=100,                         # when to save checkpoint
    # save_total_limit=2,                     # limit the total amount of checkpoints
    eval_strategy="steps",                  # evaluate every n steps
    eval_steps=100,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    model_init_kwargs=None,
    max_length=1024,
    max_prompt_length=300,
    beta=0.1,                               # same as huggingface documentation, default value
    loss_type="sigmoid",                    # default value in huggingface documentation
)

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train()

### 3. Remove tf32

Does it train faster? No, but memory consumption is half. Thus I can train on a single GPU or double the batch size.

If I train on a single gpu the training is faster, each iteration takes 18-20 seconds instead of 26. This happens because it avoids the comunication between gpus and the gpu is all the time working.

I have tried using data parallelism but does not work.

```python
import torch
model = torch.nn.DataParallel(model)
```

In [ ]:
# LoRA config based on https://github.com/ironbar/prompt_recovery/blob/main/notebooks/014_fine-tune_mistral_v2.ipynb
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05, # 0.1, althought Vaca suggested to use 0.05 for big models
        # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters
        r=16,
        bias="none",
        # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# https://www.philschmid.de/dpo-align-llms-in-2024-with-trl
# https://huggingface.co/docs/trl/main/en/dpo_trainer
# https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments
args = DPOConfig(
    output_dir="/mnt/hdd0/Kaggle/aimo/experiments/18_dpo/03_4_epochs",               # directory to save and repository id
    num_train_epochs=4,                     # number of training epochs
    per_device_train_batch_size=2,         # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    per_device_eval_batch_size=2,           # batch size for evaluation
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=2e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=10,
    save_steps=100,                         # when to save checkpoint
    # save_total_limit=2,                     # limit the total amount of checkpoints
    eval_strategy="steps",                  # evaluate every n steps
    eval_steps=100,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    # tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    model_init_kwargs=None,
    max_length=1024,
    max_prompt_length=300,
    beta=0.1,                               # same as huggingface documentation, default value
    loss_type="sigmoid",                    # default value in huggingface documentation
)

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train()

### 4. Use V1 dataset that has python block on prompt

In [16]:
# LoRA config based on https://github.com/ironbar/prompt_recovery/blob/main/notebooks/014_fine-tune_mistral_v2.ipynb
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05, # 0.1, althought Vaca suggested to use 0.05 for big models
        # r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters
        r=16,
        bias="none",
        # target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# https://www.philschmid.de/dpo-align-llms-in-2024-with-trl
# https://huggingface.co/docs/trl/main/en/dpo_trainer
# https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments
args = DPOConfig(
    output_dir="/mnt/hdd0/Kaggle/aimo/experiments/18_dpo/06_v1_dataset",               # directory to save and repository id
    num_train_epochs=4,                     # number of training epochs
    per_device_train_batch_size=2,         # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    per_device_eval_batch_size=2,           # batch size for evaluation
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=2e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_steps=50,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=10,
    save_steps=100,                         # when to save checkpoint
    # save_total_limit=2,                     # limit the total amount of checkpoints
    eval_strategy="steps",                  # evaluate every n steps
    eval_steps=100,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    # tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    model_init_kwargs=None,
    max_length=1024,
    max_prompt_length=300,
    beta=0.1,                               # same as huggingface documentation, default value
    loss_type="sigmoid",                    # default value in huggingface documentation
)

trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:411: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/10217 [00:00<?, ? examples/s]

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

  0%|          | 0/2552 [00:00<?, ?it/s]

/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


...

  0%|          | 0/222 [00:00<?, ?it/s]

{'eval_loss': 0.9329873919487, 'eval_runtime': 241.5419, 'eval_samples_per_second': 1.838, 'eval_steps_per_second': 0.919, 'eval_rewards/chosen': -6.304708003997803, 'eval_rewards/rejected': -9.430541038513184, 'eval_rewards/accuracies': 0.727477490901947, 'eval_rewards/margins': 3.12583327293396, 'eval_logps/rejected': -184.7933349609375, 'eval_logps/chosen': -136.70143127441406, 'eval_logits/rejected': -3.997708320617676, 'eval_logits/chosen': -5.640864372253418, 'epoch': 2.82}


/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/gbarbadillo/data/deepseekmath - will assume that the vocabulary was not modified.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/gbarbadillo/miniconda3/envs/aimo/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'loss': 0.009, 'grad_norm': 0.2490234375, 'learning_rate': 4.03513065434528e-06, 'rewards/chosen': -1.7783300876617432, 'rewards/rejected': -13.076261520385742, 'rewards/accuracies': 0.9937499761581421, 'rewards/margins': 11.297931671142578, 'logps/rejected': -223.69815063476562, 'logps/chosen': -88.68041229248047, 'logits/rejected': -4.250003814697266, 'logits/chosen': -5.50979471206665, 'epoch': 2.83}
{'loss': 0.0084, 'grad_norm': 0.142578125, 'learning_rate': 3.934823580888473e-06, 'rewards/chosen': -1.712306261062622, 'rewards/rejected': -11.985713958740234, 'rewards/accuracies': 0.9937499761581421, 'rewards/margins': 10.273408889770508, 'logps/rejected': -202.80624389648438, 'logps/chosen': -87.18635559082031, 'logits/rejected': -3.3614087104797363, 'logits/chosen': -4.8861589431762695, 'epoch': 2.85}
{'loss': 0.0045, 'grad_norm': 0.036865234375, 'learning_rate': 3.83547273853638e-06, 'rewards/chosen': -1.8169723749160767, 'rewards/rejected': -12.242486000061035, 'rewards/accurac

KeyboardInterrupt: 

## TODO

- [x] Count the tokens in the train set, max length and max prompt length
- [x] Do the training parameters from the example have sense?
- [x] Shuffle the data?
- [x] Verify that the model generates text correctly, on the prompt recovery challenge after fine-tuning the model forgot to use the eos token.